```python
from google.colab import drive
drive.mount('/content/drive')
import os
DIR = "/content/drive/MyDrive/Colab Notebooks/MNA/TC4029 - Ciencia y analítica de datos/Semana 6/Actividad6_FE"
os.chdir(DIR)
```

In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import json
from sklearn.metrics import mean_squared_error
from math import sqrt
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds
import time
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
from sklearn.preprocessing import MinMaxScaler

# Steam user items

In [44]:
steam_users = pd.read_json('steam_users_p3.json', lines=True)

In [4]:
working_df = steam_users.explode('items')
user_items_df = pd.json_normalize(working_df['items'])
user_items_df = user_items_df.join(working_df[['user_id', 'user_url']].reset_index(drop=True))


# Steam australian reviews.

In [51]:
steam_aus_rev_df = pd.read_json('MNA_GVD24/steam_aus_rev_p3.json', lines=True)

In [48]:
working_df = steam_aus_rev_df.explode('reviews')
reviews_df = pd.json_normalize(working_df['reviews'])
reviews_df = reviews_df.join(working_df[['user_id', 'user_url']].reset_index(drop=True))

/var/folders/5n/lp3_lp252c17c644s2777gmr0000gn/T/ipykernel_44616/648291562.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  steam_games_df['item_id'] =  pd.to_numeric(steam_games_df['item_id'], downcast='integer')


# Steam games df 

In [53]:
steam_games_df = pd.read_json('steam_games_p3.json', lines=True)
steam_games_df = steam_games_df.rename(columns={'id': 'item_id'})

In [54]:
steam_games_df = steam_games_df.drop(columns=['metascore'], axis=1)
steam_games_df['publisher'].fillna('', inplace=True)
steam_games_df['genres'].fillna('[]', inplace=True)  
steam_games_df['tags'].fillna('[]', inplace=True) 

,funny,posted,last_edited,item_id,helpful,recommend,review,user_id,user_url
0,,"Posted November 5, 2011.",,1250.0,No ratings yet,1,Simple yet with great replayability. In my opi...,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,,"Posted July 15, 2011.",,22200.0,No ratings yet,1,It's unique and worth a playthrough.,76561197970982479,http://steamcommunity.com/profiles/76561197970...
2,,"Posted April 21, 2011.",,43110.0,No ratings yet,1,Great atmosphere. The gunplay can be a bit chu...,76561197970982479,http://steamcommunity.com/profiles/76561197970...
3,,"Posted June 24, 2014.",,251610.0,15 of 20 people (75%) found this review helpful,1,I know what you think when you see this title ...,js41637,http://steamcommunity.com/id/js41637
4,,"Posted September 8, 2013.",,227300.0,0 of 1 people (0%) found this review helpful,1,For a simple (it's actually not all that simpl...,js41637,http://steamcommunity.com/id/js41637
...,...,...,...,...,...,...,...,...,...
59300,,Posted July 10.,,70.0,No ratings yet,1,a must have classic from steam definitely wort...,76561198312638244,http://steamcommunity.com/profiles/76561198312...
59301,,Posted July 8.,,362890.0,No ratings yet,1,this game is a perfect remake of the original ...,76561198312638244,http://steamcommunity.com/profiles/76561198312...
59302,1 person found this review funny,Posted July 3.,,273110.0,1 of 2 people (50%) found this review helpful,1,had so much fun plaing this and collecting res...,LydiaMorley,http://steamcommunity.com/id/LydiaMorley
59303,,Posted July 20.,,730.0,No ratings yet,1,:D,LydiaMorley,http://steamcommunity.com/id/LydiaMorley


In [ ]:
steam_games_df.to_json('steam_games_df.json', orient='records', lines=True)
reviews_df.to_json('reviews_df.json', orient='records', lines=True)
user_items_df.to_json('user_items_df.json', orient='records', lines=True)
